In [4]:
!cp "/content/drive/MyDrive/2021_2_class/Cat_and_dog.tar" "/content/data/Cat_and_dog.tar"

cp: cannot stat '/content/drive/MyDrive/2021_2_class/Cat_and_dog.tar': No such file or directory


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!tar -xvf "/content/data/Cat_and_dog.tar"

tar: /content/data/Cat_and_dog.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
# -*- coding:utf-8 -*-
from random import shuffle

import tensorflow as tf
import numpy as np
import easydict # 저는 이 기능을 전역변수로서 사용하려고 합니다.
import matplotlib.pyplot as plt
import os

FLAGS = easydict.EasyDict({"img_size": 224,
                           
                           "tr_txt_path": "/content/train.txt",

                           "tr_img_path": "/content/Cat_Dog_500_train/",

                           "val_txt_path": "/content/val.txt",

                           "val_img_path": "/content/Cat_Dog_100_val/",

                           "te_txt_path": "/content/test.txt",

                           "te_img_path": "/content/Cat_Dog_200_test/",
                           
                           "batch_size": 24,

                           "pre_checkpoint": False, # 학습이 멈추어 이어서 시작해야 할때, 혹은 테스트를 진행할 때

                           "pre_checkpoint_path": "",   # 학습된 weight가 저장된 경로

                           "save_checkpoint": "/content/drive/MyDrive/2021_2_class/checkpoint",
                           
                           "epochs": 10,
                           
                           "lr": 0.001})

optim = tf.keras.optimizers.Adam(FLAGS.lr)

def tr_input(img_list, lab_list):

    img = tf.io.read_file(img_list)
    img = tf.image.decode_jpeg(img, 3)
    img = tf.image.resize(img, [FLAGS.img_size, FLAGS.img_size]) / 255.

    lab = lab_list

    return img, lab

def cal_loss(model, images, labels):

    with tf.GradientTape() as tape:

        logits = model(images, True)

        loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(labels, logits)

    grads = tape.gradient(loss, model.trainable_variables)
    # 첫번 째 conv 에서의 grads의 모양은 ==> [7, 7, 3, 64] ==> 7x7x3x64
    optim.apply_gradients(zip(grads, model.trainable_variables))
    # updataed weight = previou weight - (lr x grads)
    # a = [1,2,3], b = [4,5,6] ==> a와 b를 zip하면 ==> [[1,4], [2,5], [3,6]]


    return loss

def main():
    
    # 1. 모델 정의
    model = tf.keras.applications.VGG16(include_top=False, input_shape=(FLAGS.img_size, FLAGS.img_size, 3))
    x = model.output    # [B, 7, 7, 512]
    h = tf.keras.layers.Flatten()(x)
    h = tf.keras.layers.Dense(1024)(h)
    h = tf.keras.layers.ReLU()(h)
    h = tf.keras.layers.Dense(1024)(h)
    h = tf.keras.layers.Dense(1)(h)
    model = tf.keras.Model(inputs= model.input, outputs=h)

    model.summary()

    if FLAGS.pre_checkpoint:
        ckpt = tf.train.Checkpoint(model=model, optim=optim)
        ckpt_manager = tf.train.CheckpointManager(ckpt, FLAGS.pre_checkpoint_path, 5)

        if ckpt_manger.latest_checkpoint:
            ckpt.restore(ckpt_manger.latest_checkpoint)
            print("Restored!!!!!")

    # 2. 입력 정의 (학습, 테스트, 확인용)
    tr_img_data = np.loadtxt(FLAGS.tr_txt_path, dtype="<U100", skiprows=0, usecols=0)
    tr_img_data = [FLAGS.tr_img_path + data for data in tr_img_data]
    tr_img_data = np.array(tr_img_data)
    tr_lab_data = np.loadtxt(FLAGS.tr_txt_path, dtype=np.int32, skiprows=0, usecols=1)

    val_img_data = np.loadtxt(FLAGS.val_txt_path, dtype="<U100", skiprows=0, usecols=0)
    val_img_data = [FLAGS.val_img_path + data for data in val_img_data]
    val_img_data = np.array(val_img_data)
    val_lab_data = np.loadtxt(FLAGS.val_txt_path, dtype=np.int32, skiprows=0, usecols=1)

    te_img_data = np.loadtxt(FLAGS.te_txt_path, dtype="<U100", skiprows=0, usecols=0)
    te_img_data = [FLAGS.te_img_path + data for data in te_img_data]
    te_img_data = np.array(te_img_data)
    te_lab_data = np.loadtxt(FLAGS.te_txt_path, dtype=np.int32, skiprows=0, usecols=1)

    count = 0
    for epoch in range(FLAGS.epochs):

        tr_generator = tf.data.Dataset.from_tensor_slices((tr_img_data, tr_lab_data))
        tr_generator = tr_generator.shuffle(len(tr_img_data))
        tr_generator = tr_generator.map(tr_input)
        tr_generator = tr_generator.batch(FLAGS.batch_size)
        tr_generator = tr_generator.prefetch(tf.data.experimental.AUTOTUNE)

        tr_iter = iter(tr_generator)
        tr_idx = len(tr_img_data) // FLAGS.batch_size
        for step in range(tr_idx):
            batch_images, batch_labels = next(tr_iter)

            loss = cal_loss(model, batch_images, batch_labels)

            if count % 10 == 0:
                print("Epoch: {} [{}/{}] loss = {}".format(epoch, step + 1, tr_idx, loss))
            

            if count % 200 == 0:
                num = str(int(count) // 200)
                model_dir = "%s/%s" % (FLAGS.save_checkpoint, num)

                ckpt = tf.train.Checkpoint(model=model, optim=optim)
                if not os.path.isdir(model_dir):
                    os.makedirs(model_dir)
                    print("Made {} files to save weights".format(num))

                ckpt_dir = model_dir + "/" + "fix_vgg_16_model_{}.ckpt".format(count)
                ckpt.save(ckpt_dir)



            count += 1





    # Train, Test, Validation
 
if __name__ == "__main__":
    main()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   